**Created by Sanjay Raghuwanshi ** <br/>
[Github](https://github.com/sanjay-raghu) <br/>
[Linkedin](https://www.linkedin.com/in/sanjayiitg/) <br/>
**Sentiment Analysis:** the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.

**What's New** I have added how to deal with data imbalance. Almost all classification task have this problem as number of data of every class if different. For current dataset number of data having positive sentiments is very low relative to data with negative sentiment.

**Solving class imbalaned data**:
- upsampling
- using class weighted loss function

Using LSTM to classify the movie reviews into positive and negative.


In [ ]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Only keeping the necessary columns.

In [ ]:
data = pd.read_csv('california_housing_test.csv')
# Keeping only the neccessary columns
#data = data[['Username','Comment']]
#data.dtypes
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


Data preview

In [ ]:
data.head()

,post.ID,Caption
0,3.11046E+18,Berdasarkan survei elektabilitas capres yang d...
1,Username,Comment
2,post.ID,Caption
3,3.11046E+18,Berdasarkan survei elektabilitas capres yang d...
4,Username,Comment


In [ ]:
data.dropna()

,post.ID,Caption
0,3.11046E+18,Berdasarkan survei elektabilitas capres yang d...
1,Username,Comment
2,post.ID,Caption
3,3.11046E+18,Berdasarkan survei elektabilitas capres yang d...
4,Username,Comment
...,...,...
224,misjuhana,Kurang memukau seperti nya 😬
225,therifkimaul,pange pange..
226,silalahibrevard,Kerjanya tidak ada yg beres. Lihat Jakarta 🙌
227,tsabitqaish,Hahahha survei abal abal 😂


Next, I am dropping the 'Neutral' sentiments as my goal was to only differentiate positive and negative tweets. After that, I am filtering the tweets so only valid texts and words remain.  Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [ ]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
# removing special chars
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#
data.head()

<ipython-input-7-f8f03ba23afd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())


,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative


In [ ]:
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt','')
data.head()

2330
8426


,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive
5,gregabbott_tx tedcruz on my first day i will ...,Positive
6,warriorwoman91 i liked her and was happy when...,Negative


In [ ]:

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  272,
          96,    2,  574,    1,   16,   26,  282,   29,  136,    5,  139,
        1309,    9,  975,  976,  637],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,   15,  201,
         216,    6,  812,   75,  109,   25, 1953,   97,    5,    2,  187,
           9,    1,  143,  474,   12]], dtype=int32)

Next, I compose the LSTM Network. Note that **embed_dim**, **lstm_out**, **batch_size**, **droupout_x** variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 27, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Hereby I declare the train and test dataset.

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4302, 27) (4302, 2)
(1076, 27) (1076, 2)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

Epoch 1/15
34/34 [==============================] - 10s 216ms/step - loss: 0.5419 - accuracy: 0.7780
Epoch 2/15
34/34 [==============================] - 7s 216ms/step - loss: 0.4558 - accuracy: 0.7952
Epoch 3/15
34/34 [==============================] - 7s 216ms/step - loss: 0.3426 - accuracy: 0.8538
Epoch 4/15
34/34 [==============================] - 7s 221ms/step - loss: 0.2725 - accuracy: 0.8854
Epoch 5/15
34/34 [==============================] - 7s 196ms/step - loss: 0.2331 - accuracy: 0.9021
Epoch 6/15
34/34 [==============================] - 7s 215ms/step - loss: 0.2178 - accuracy: 0.9077
Epoch 7/15
34/34 [==============================] - 6s 190ms/step - loss: 0.2040 - accuracy: 0.9182
Epoch 8/15
34/34 [==============================] - 7s 215ms/step - loss: 0.1681 - accuracy: 0.9324
Epoch 9/15
34/34 [==============================] - 7s 200ms/step - loss: 0.1407 - accuracy: 0.9410
Epoch 10/15
34/34 [==============================] - 7s 203ms/step - loss: 0.1207 - accuracy: 0.954

Extracting a validation set, and measuring score and accuracy.

In [ ]:
predict_x=model.predict(X_test)
Y_pred=np.argmax(predict_x,axis=1)
#Y_pred = model.predict_classes(X_test,batch_size = batch_size)

34/34 [==============================] - 1s 12ms/step


In [ ]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[725  87]
 [139 125]]
              precision    recall  f1-score   support

           0       0.84      0.89      0.87       812
           1       0.59      0.47      0.53       264

    accuracy                           0.79      1076
   macro avg       0.71      0.68      0.70      1076
weighted avg       0.78      0.79      0.78      1076



Finally measuring the number of correct guesses.  It is clear that finding negative tweets (**class 0**) goes very well (**recall 0.92**) for the Network but deciding whether is positive (**class 1**) is not really (**recall 0.52**). My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

As expected accuracy for positive data is vary low compare to negative, Lets try to solve this problem.

**1. Up-sample Minority Class**

Up-sampling is the process of randomly duplicating observations from the minority class in order to reinforce its signal.
There are several heuristics for doing so, but the most common way is to simply resample with replacement.

In [ ]:
# Separate majority and minority classes
data_majority = data[data['sentiment'] == 'Negative']
data_minority = data[data['sentiment'] == 'Positive']

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

In [ ]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())


positive data in training: 1789
negative data in training: 6794
positive data in test: 447
negative data in test: 1699


In [ ]:
# Separate majority and minority classes in training data for upsampling
data_majority = train[train['sentiment'] == 'Negative']
data_minority = train[train['sentiment'] == 'Positive']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority,
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])

# Display new class counts
print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['text'].values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['sentiment']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

majority class before upsample: (6794, 2)
minority class before upsample: (1789, 2)
After upsampling
Negative    6794
Positive    6794
Name: sentiment, dtype: int64
x_train shape: (13588, 29)
x_test shape (2146, 29)


In [ ]:
# model
embed_dim = 128
lstm_out = 192

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 29, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 29, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 192)               246528    
                                                                 
 dense_1 (Dense)             (None, 2)                 386       
                                                                 
Total params: 502,914
Trainable params: 502,914
Non-trainable params: 0
_________________________________________________________________
None


Here we train the Network. We should run much more than 15 epoch, but I would have to wait forever for kaggle, so it is 15 for now.

In [ ]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1.6/bias }
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/15
107/107 [==============================] - 43s 376ms/step - loss: 1.2021 - accuracy: 0.5865
Epoch 2/15
107/107 [==============================] - 42s 386ms/step - loss: 0.7464 - accuracy: 0.7761
Epoch 3/15
107/107 [==============================] - 40s 375ms/step - loss: 0.6160 - accuracy: 0.8265
Epoch 4/15
107/107 [==============================] - 40s 372ms/step - loss: 0.5455 - accuracy: 0.8506
Epoch 5/15
107/107 [==============================] - 40s 373ms/step - loss: 0.4888 - accuracy: 0.8664
Epoch 6/15
107/107 [==============================] - 40s 376ms/step - loss: 0.4396 - accuracy: 0.8845
Epoch 7/15
107/107 [==============================] - 41s 387ms/step - loss: 0.4266 - accuracy: 0.8862
Epoch 8/15
107/107 [==============================] - 40s 375ms/step - loss: 0.3908 - accuracy: 0.8970
Epoch 9/15
107/107 [==============================] - 40s 377ms/step - loss: 0.3721 - accuracy: 0.9032
Epoch 10/15
107/107 [==============================] - 40s 375ms/step - l

In [ ]:
predict_x=model.predict(X_test)
Y_pred=np.argmax(predict_x,axis=1)
#Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

68/68 [==============================] - 2s 21ms/step
confusion matrix [[1332  367]
 [ 116  331]]
              precision    recall  f1-score   support

           0       0.92      0.78      0.85      1699
           1       0.47      0.74      0.58       447

    accuracy                           0.77      2146
   macro avg       0.70      0.76      0.71      2146
weighted avg       0.83      0.77      0.79      2146



So the class imbalance is reduced significantly recall value for positive tweets (Class 1) improved from 0.54 to 0.77. It is alwayes not possible to reduce it compleatly.

You may also noticed that the recall value for Negative tweets also decreased from 0.90 to 0.78  but this can be improved using training model to more epocs and tuning the hyperparameters.

In [ ]:
# running model to few more epochs
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)
predict_x=model.predict(X_test)
Y_pred=np.argmax(predict_x,axis=1)
#Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

Epoch 1/15
107/107 [==============================] - 40s 374ms/step - loss: 0.2862 - accuracy: 0.9285
Epoch 2/15
107/107 [==============================] - 40s 374ms/step - loss: 0.2787 - accuracy: 0.9303
Epoch 3/15
107/107 [==============================] - 40s 375ms/step - loss: 0.2673 - accuracy: 0.9341
Epoch 4/15
107/107 [==============================] - 40s 375ms/step - loss: 0.2691 - accuracy: 0.9327
Epoch 5/15
107/107 [==============================] - 43s 398ms/step - loss: 0.2740 - accuracy: 0.9327
Epoch 6/15
107/107 [==============================] - 41s 377ms/step - loss: 0.2580 - accuracy: 0.9344
Epoch 7/15
107/107 [==============================] - 40s 378ms/step - loss: 0.2548 - accuracy: 0.9362
Epoch 8/15
107/107 [==============================] - 41s 379ms/step - loss: 0.2543 - accuracy: 0.9357
Epoch 9/15
107/107 [==============================] - 41s 381ms/step - loss: 0.2442 - accuracy: 0.9376
Epoch 10/15
107/107 [==============================] - 41s 380ms/step - l

In [ ]:
twt = ['keep up the good work']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 381  47   1 137 464]]
1/1 - 0s - 35ms/epoch - 35ms/step
positive


1. ## Tuning the hyper-parameters using gridsearch

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# # Function to create model, required for KerasClassifier
# def create_model(dropout_rate = 0.0):
#     # create model
#     embed_dim = 128
#     lstm_out = 192
#     model = Sequential()
#     model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
#     model.add(SpatialDropout1D(dropout_rate))
#     model.add(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=dropout_rate))
#     model.add(Dense(2,activation='softmax'))
#     model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# #     print(model.summary())
#     return model
# # fix random seed for reproducibility
# seed = 7
# np.random.seed(seed)

# model = KerasClassifier(build_fn=create_model, verbose=2,epochs=30, batch_size=128)
# # define the grid search parameters
# # batch_size = [128]
# # epochs = [5]
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# # class_weight = [{0: 1, 1: 1/bias},{0: 1, 1: 1.2/bias},{0: 1, 1: 1.5/bias},{0: 1, 1: 1.8/bias}]
# param_grid = dict(dropout_rate = dropout_rate)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# grid_result = grid.fit(X_train, Y_train)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Finally measuring the number of correct guesses.  It is clear that finding negative tweets goes very well for the Network but deciding whether is positive is not really. My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

its 10% increase in positive accuracy from 56% to 66%

In [ ]:
twt = ['inaccurate facts, dont vote for him']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0 1288   43  200   14
    74]]
1/1 - 0s - 28ms/epoch - 28ms/step
negative
